In [6]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np

In [7]:
## Acessando pelo pacote da BD

df=bd.read_sql("SELECT * FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_publicos`", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

Downloading: 100%|██████████| 469/469 [00:00<00:00, 1424.17rows/s]


In [5]:
df['ano'].unique()

## tenho de 2004 a 2021:



<IntegerArray>
[2016, 2010, 2012, 2009, 2018, 2006, 2008, 2004, 2017, 2005, 2015, 2011, 2019,
 2013, 2020, 2003, 2021, 2014, 2007]
Length: 19, dtype: Int64

In [8]:
### Criando função para deflacionar a variável de remuneração
### Vou me basear no IPCA acumulado de dezembro de 2022 para realização do cálculo

## adicionar coluna de fator de correção pra cada ano
## depois faz a conta

os.chdir("G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\DADOS\\415 - Repositório de Dados\\Documentos auxiliares")

In [19]:
ipca = pd.read_csv("ipca.csv",sep=";",decimal=",")

In [21]:
ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ano                     28 non-null     int64  
 1   IPCA acumulado dez/ano  28 non-null     float64
 2   It                      28 non-null     float64
 3   Fator de correcao       28 non-null     float64
dtypes: float64(3), int64(1)
memory usage: 1.0 KB


In [22]:
df1= df.merge(ipca[['Ano','Fator de correcao']], right_on='Ano',left_on='ano')

0      1,355857127
1      1,355857127
2      1,355857127
3      1,355857127
4      1,355857127
          ...     
464    2,370358141
465    2,370358141
466    2,370358141
467    2,370358141
468    2,370358141
Name: Fator de correcao, Length: 469, dtype: object

In [23]:
df1['media_remuneracao_ajustada']= df1['media_remuneracao']* df1['Fator de correcao']

In [24]:
df1

,ano,variavel,categoria,media_remuneracao,Ano,Fator de correcao,media_remuneracao_ajustada
0,2016,genero,Feminino,3648.047669,2016,1.355857,4946.231432
1,2016,genero,Masculino,5145.268336,2016,1.355857,6976.248744
2,2016,poderes,Outros,6589.952686,2016,1.355857,8935.034316
3,2016,poderes,Executivo,3252.946308,2016,1.355857,4410.530435
4,2016,poderes,Judiciário,11438.431550,2016,1.355857,15508.878939
...,...,...,...,...,...,...,...
464,2007,grau_instrucao,Até Mestrado,4042.200402,2007,2.370358,9581.462632
465,2007,grau_instrucao,Até Doutorado,6958.183786,2007,2.370358,16493.387584
466,2007,grau_instrucao,Até Fundamental,994.996071,2007,2.370358,2358.497038
467,2007,grau_instrucao,Até Ensino Médio,1432.581737,2007,2.370358,3395.731784


In [25]:
## Subindo para datalake
client = bigquery.Client()
dataset_ref = client.dataset('perfil_remuneracao')



In [30]:
df1 = df1[['ano', 'variavel', 'categoria', 'media_remuneracao',
       'Fator de correcao', 'media_remuneracao_ajustada']]

In [32]:
df1=df1.rename(columns={'Fator de correcao':'fator_correcao'})

In [34]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469 entries, 0 to 468
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ano                         469 non-null    Int64  
 1   variavel                    469 non-null    object 
 2   categoria                   469 non-null    object 
 3   media_remuneracao           469 non-null    float64
 4   fator_correcao              469 non-null    float64
 5   media_remuneracao_ajustada  469 non-null    float64
dtypes: Int64(1), float64(3), object(2)
memory usage: 26.1+ KB


In [35]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência'),
 bigquery.SchemaField('variavel','STRING',description='De qual variável aquela remuneração se refere'),
 bigquery.SchemaField('categoria','STRING',description='Categoria dentro daquela variável'),
 bigquery.SchemaField('media_remuneracao','FLOAT',description='Valor nominal da média de remuneração daquele ano'),
bigquery.SchemaField('fator_correcao','FLOAT',description='índice para correção monetária baseando-se no IPCA para dezembro de 2022'),
bigquery.SchemaField('media_remuneracao_ajustada','FLOAT',description='Valor nominal da média de remuneração daquele ano ajustado para dezembro de 2022')
 ]


In [36]:
table_ref = dataset_ref.table('RAIS_remuneracao_vinculos_publicos')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=b1035481-9e11-4297-a378-2c91782606dd>